In [57]:
import os
import numpy as np
import pandas as pd
from glob import glob
import random

# === CONFIGURATION ===
input_dir = r"C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\samples augment\real groups\layout 1"  # Folder containing G1_layout1.csv ... G10_layout1.csv
output_csv = r"C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\weighted combination\layout 1\G15\augmented_G15_L1.csv"
frames_per_sample = 800
samples_per_class = 50
num_actions = 5
coordinates = ["1,1.5", "3.5,1.5", "2.75,5.1"]

# === Load and segment all data ===
all_files = glob(os.path.join(input_dir, "*.csv"))
print(f"Found {len(all_files)} CSV files.")

class_samples = {}

for file in all_files:
    print(f"\n🔍 Processing file: {os.path.basename(file)}")
    try:
        df = pd.read_csv(file)
        print(f"Loaded {len(df)} rows.")

        df["Coordinates"] = df["Coordinates"].astype(str).str.replace(" ", "").str.strip()
        df["Action"] = df["Action"].astype(int)

        total_rows = len(df)
        num_samples = total_rows // frames_per_sample
        print(f"Found {num_samples} complete samples.")

        for i in range(num_samples):
            start = i * frames_per_sample
            end = start + frames_per_sample
            sample = df.iloc[start:end]

            coord = sample["Coordinates"].iloc[0]
            action = sample["Action"].iloc[0]

            if (action, coord) not in class_samples:
                class_samples[(action, coord)] = []

            class_samples[(action, coord)].append(sample)

    except Exception as e:
        print(f"❌ Error loading {file}: {e}")

# === Print class-wise sample counts ===
print("\n📊 Sample counts per (action, coordinate):")
for key, samples in class_samples.items():
    print(f"  {key}: {len(samples)} samples")

# Final fixed augmentation loop
augmented_samples = []
sample_counter = {}

for (action, coord), samples in class_samples.items():
    if len(samples) < 2:
        print(f"⚠️ Not enough samples for class ({action}, {coord}). Skipping.")
        continue

    print(f"✅ Augmenting class ({action}, {coord}) with {samples_per_class} samples...")
    for i in range(samples_per_class):
        try:
            s1, s2 = random.sample(samples, 2)

            numeric_cols = s1.select_dtypes(include=[np.number]).columns
            s1_values = s1[numeric_cols].values
            s2_values = s2[numeric_cols].values

            w1 = np.random.uniform(0.2, 0.8)
            w2 = 1 - w1
            combined_values = w1 * s1_values + w2 * s2_values

            combined_df = s1.copy()
            combined_df[numeric_cols] = combined_values
            combined_df["Coordinates"] = coord
            combined_df["Action"] = action

            key = (action, coord)
            sample_num = sample_counter.get(key, 1)
            combined_df["Sample"] = sample_num
            sample_counter[key] = sample_num + 1

            augmented_samples.append(combined_df)

        except Exception as aug_e:
            print(f"❌ Error during augmentation: {aug_e}")
            
# === Save if any samples were created ===
if augmented_samples:
    final_df = pd.concat(augmented_samples, axis=0).reset_index(drop=True)
    final_df.to_csv(output_csv, index=False)
    print(f"\n✅ Augmented CSV saved with {len(final_df)} rows at:")
else:
    print("\n❌ No augmented samples generated.")

Found 8 CSV files.

🔍 Processing file: csi_all_samples_G1_L1.csv
Loaded 600015 rows.
Found 750 complete samples.

🔍 Processing file: csi_all_samples_G2_L1.csv
Loaded 600015 rows.
Found 750 complete samples.

🔍 Processing file: csi_all_samples_G3_L1.csv
Loaded 600013 rows.
Found 750 complete samples.

🔍 Processing file: csi_all_samples_G4_L1.csv
Loaded 600015 rows.
Found 750 complete samples.

🔍 Processing file: csi_all_samples_G5_L1.csv
Loaded 600649 rows.
Found 750 complete samples.

🔍 Processing file: csi_all_samples_G6_l1.csv
Loaded 600014 rows.
Found 750 complete samples.

🔍 Processing file: csi_all_samples_G7_L1.csv
Loaded 600015 rows.
Found 750 complete samples.

🔍 Processing file: csi_all_samples_G9_L1.csv
Loaded 600014 rows.
Found 750 complete samples.

📊 Sample counts per (action, coordinate):
  (1, '1,1.5'): 408 samples
  (1, '3.5,1.5'): 400 samples
  (1, '2.75,5.1'): 400 samples
  (2, '1,1.5'): 400 samples
  (2, '3.5,1.5'): 400 samples
  (2, '2.75,5.1'): 400 samples
  (3, '1

In [58]:
import pandas as pd

# Path to your CSV file
csv_path = r"C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\weighted combination\layout 1\G15\augmented_G15_L1.csv"
output_path = r"C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\weighted combination\layout 1\G15\augmented_G15_L1_fixed.csv"

# Load full CSV
df = pd.read_csv(csv_path)

frames_per_sample = 800
num_samples = len(df) // frames_per_sample

# Fix the Frame column
corrected_frames = []

for i in range(num_samples):
    corrected_frames.extend(range(1, frames_per_sample + 1))

# If any leftover rows (incomplete sample), retain original values
remainder = len(df) % frames_per_sample
if remainder:
    corrected_frames.extend(df["Frame"].iloc[-remainder:].tolist())

# Replace and save
df["Frame"] = corrected_frames
df.to_csv(output_path, index=False)

print(f"✅ Frame column fixed and saved to: {output_path}")


✅ Frame column fixed and saved to: C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\weighted combination\layout 1\G15\augmented_G15_L1_fixed.csv


In [ ]:
# === CONFIGURABLE SCALING FACTORS ===
amp_scaling_factor = 0.9     # e.g., 0.9 means reduce amplitude
phase_scaling_factor = 1.1   # e.g., 1.1 means increase phase

import os
import numpy as np
import pandas as pd
from glob import glob
import random

# === CONFIGURATION ===
input_dir = r"C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\samples augment\real groups\layout 1\csi_all_samples"  # Folder containing G1_layout1.csv ... G10_layout1.csv
output_csv = r"C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\weighted combination\layout 1\G15\augmented_G15_L1.csv"
frames_per_sample = 800
samples_per_class = 50
num_actions = 5
coordinates = ["1,1.5", "3.5,1.5", "2.75,5.1"]

# === Load and segment all data ===
all_files = glob(os.path.join(input_dir, "*.csv"))
print(f"Found {len(all_files)} CSV files.")

class_samples = {}

for file in all_files:
    print(f"\n🔍 Processing file: {os.path.basename(file)}")
    try:
        df = pd.read_csv(file)
        print(f"Loaded {len(df)} rows.")

        df["Coordinates"] = df["Coordinates"].astype(str).str.replace(" ", "").str.strip()
        df["Action"] = df["Action"].astype(int)

        total_rows = len(df)
        num_samples = total_rows // frames_per_sample
        print(f"Found {num_samples} complete samples.")

        for i in range(num_samples):
            start = i * frames_per_sample
            end = start + frames_per_sample
            sample = df.iloc[start:end]

            coord = sample["Coordinates"].iloc[0]
            action = sample["Action"].iloc[0]

            if (action, coord) not in class_samples:
                class_samples[(action, coord)] = []

            class_samples[(action, coord)].append(sample)

    except Exception as e:
        print(f"❌ Error loading {file}: {e}")

# === Print class-wise sample counts ===
print("\n📊 Sample counts per (action, coordinate):")
for key, samples in class_samples.items():
    print(f"  {key}: {len(samples)} samples")


# Final fixed augmentation loop with scaling
augmented_samples = []
sample_counter = {}

for (action, coord), samples in class_samples.items():
    if len(samples) < 2:
        print(f"⚠️ Not enough samples for class ({action}, {coord}). Skipping.")
        continue

    print(f"✅ Augmenting class ({action}, {coord}) with {samples_per_class} samples...")
    for i in range(samples_per_class):
        try:
            s1, s2 = random.sample(samples, 2)

            numeric_cols = s1.select_dtypes(include=[np.number]).columns
            s1_values = s1[numeric_cols].values
            s2_values = s2[numeric_cols].values

            w1 = np.random.uniform(0.2, 0.8)
            w2 = 1 - w1
            combined_values = w1 * s1_values + w2 * s2_values

            # Apply scaling
            amp_cols = [col for col in numeric_cols if col.startswith("Amp_dBm[")]
            phase_cols = [col for col in numeric_cols if col.startswith("Phase_deg[")]

            amp_indices = [numeric_cols.get_loc(c) for c in amp_cols]
            phase_indices = [numeric_cols.get_loc(c) for c in phase_cols]

            combined_values[:, amp_indices] *= amp_scaling_factor
            combined_values[:, phase_indices] *= phase_scaling_factor

            # Build final DataFrame
            combined_df = s1.copy()
            combined_df[numeric_cols] = combined_values
            combined_df["Coordinates"] = coord
            combined_df["Action"] = action

            key = (action, coord)
            sample_num = sample_counter.get(key, 1)
            combined_df["Sample"] = sample_num
            sample_counter[key] = sample_num + 1

            augmented_samples.append(combined_df)

        except Exception as aug_e:
            print(f"❌ Error during augmentation: {aug_e}")


In [19]:
import os
import pandas as pd
import numpy as np
from glob import glob

# === CONFIGURATION ===
input_file =  r"C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\samples\G9\layout 1\csi_all_samples_G9_L1.csv"          # Folder with G1_L1.csv to G8_L1.csv
output_file = r"C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\amplitude phase scaling\G15\layout 1\csi_all_samples_G15_L1.csv"       # Where to save G9–G16 augmented files
amp_scaling = 0.95
phase_scaling = 1.05

# === Load and Scale ===
df = pd.read_csv(input_file)
amp_cols = [col for col in df.columns if col.startswith("Amp_dBm[")]
phase_cols = [col for col in df.columns if col.startswith("Phase_deg[")]

df[amp_cols] *= amp_scaling
df[phase_cols] *= phase_scaling

# === Save
os.makedirs(os.path.dirname(output_file), exist_ok=True)
df.to_csv(output_file, index=False)
print(f"✅ Saved scaled CSV to: {output_file}")


✅ Saved scaled CSV to: C:\Users\fdzya\Desktop\mmu\YEAR 4\fyp\resources\own dataset\amplitude phase scaling\G15\layout 1\csi_all_samples_G15_L1.csv
